In [ ]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Creamos el dataframes de las corrientes calientes

In [ ]:
Hot_datos = {0: [0,0], 1: [0,0], 2: [50,0], 3: [25,0],4: [125,190], 5:[0,0]}
Hot_df = pd.DataFrame(Hot_datos,index=['H1','H2'])
Hot_df

E igualmente de las frías.

In [ ]:
Cold_datos =  {0: [0,0], 1:[60,0], 2: [40,0], 3: [20,20], 4: [100,100], 5:[20,20]}
Cold_df = pd.DataFrame(Cold_datos,index=['C1','C2'])
Cold_df

In [ ]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
model.h=Set(initialize=hotst)
model.c=Set(initialize=coldst)
model.i=Set(initialize=Hot_df.columns)

Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [ ]:
model.pprint()

Creamos ahora las variables del modelo. Para cada tipo de utility creamos una variable para cada intervalo, luego impondremos en qué intervalos esa utility es cero.

In [ ]:
Qhps = model.Qhps = Var(model.i,within = NonNegativeReals)
Qlps = model.Qlps = Var(model.i,within = NonNegativeReals)
Qw = model.Qw = Var(model.i,within = NonNegativeReals)
R = model.R = Var(model.i,within = NonNegativeReals)

Creamos la función de coste a partir de los consumos de utilities y sus precios. Aunque las unidades no coincidan, para la solución da igual. Es decir los precios están en $/kWyr y los consumos en MW. Tendría que coger como base de cálculo un año y los MW transformarlos en kWyr. Como cada término se multiplicaría por el mismo valor, no hay problema porque la solución sería la misma (aunque no sería igual el valor de la FO).

In [ ]:
model.coste = Objective(expr = Qhps[1]*80+Qlps[3]*80 + Qw[5]*20)

Constraints

In [ ]:
ni = list(model.i)[1:] 
model.int = ConstraintList()
for i in ni:
    model.int.add(
        R[i-1]+Qhps[i]+Qlps[i]+sum(Hot_df[i]) == R[i]+sum(Cold_df[i])+Qw[i]
    )

model.R0 = Constraint(expr = R[0] == 0) 
model.R9 = Constraint(expr = R[5] == 0)

nii = list(model.i)[0:1]+list(model.i)[2:] 
model.hps = ConstraintList()
for i in nii:
    model.hps.add(
        Qhps[i]==0
    )

niii = list(model.i)[0:3]+list(model.i)[4:] 
model.lps = ConstraintList()
for i in niii:
    model.lps.add(
        Qlps[i]==0
    )

niiii = list(model.i)[0:5] 
model.cw = ConstraintList()
for i in niiii:
    model.cw.add(
        Qw[i]==0
    )

Resolvemos el modelo

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

In [ ]:
Qh = value(model.Qhps[1])
Ql = value(model.Qlps[3])
Qw = value(model.Qw[5])
print('Cold utility = {0:2.2f}, LP Steam = {1:2.2f}, HP Steam = {2:2.2f}'.format(Qw, Ql, Qh))